In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from chatbot import ChatBot, load_api_key
from OktSentiment import Sentiment
from tarot import TarotBot
from refer_text import ResponseSearch

app = Flask(__name__)
CORS(app)

# ChatBot 인스턴스 생성
api_key = load_api_key()
if not api_key:
    raise ValueError("API key not found or invalid. Please check your api_key.txt file.")
respon_search = ResponseSearch('c:/Users/SMHRD/최종_참조자료.csv',"jhgan/ko-sroberta-multitask")
chatbot = ChatBot(api_key, respon_search)  # CSV 파일 경로를 적절히 수정하세요
#C:\Users\SMHRD\Documents\카카오톡 받은 파일\원인-대처종합.csv //c:/Users/SMHRD/원인-대처종합.csv
# Sentiment 인스턴스 생성
sentiment_analyzer = Sentiment()

@app.route("/chatbot", methods=["POST"])
def chat():
    data = request.json
    print("Received data:", data)
    user_nick = data.get("user_nick")
    user_input = data.get("input")
    
    if user_input:
        try:
            response = chatbot.process_input(user_nick, user_input)
            
            if response.get("isProblem"):
                # 형태소 분석 및 감정 분석 수행
                sentiment_results = sentiment_analyzer.parse([user_input])
                # 결과가 딕셔너리 형태로 바꾸고 소수점 소수점 둘쨰자리까지
                if 'polarity' in sentiment_results and len(sentiment_results['polarity']) > 0:
                    sentiment_results = sentiment_results['polarity'][0]
                    sentiment_results = {key: round(value * 100, 2) for key, value in sentiment_results.items()}

                response.update({
                    "sentiment": sentiment_results
                })

            return jsonify(response)
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            return jsonify({"error": f"An error occurred while processing your request: {str(e)}"}), 500
    else:
        return jsonify({"error": "No input provided"}), 400

# 타로카드 라우트
@app.route("/api/interpret",methods=["POST"])

def tarot_card():
    data = request.json
    cards = data.get("cards")
    user_input = data.get("user_input")
    user_select = data.get("user_select")
    print("recived data: ",data)
    if not cards or len(cards) != 3:
        return jsonify({"error": "카드를 다시 선택해 주세요."}),400
    
    key_path = "c:/Users/SMHRD/api.txt"
    try:
        tarot = TarotBot(api_key_path=key_path)
        if user_input:
            print(user_input)
            response = tarot.interpret_cards(user_input,cards)
        else:
            response = tarot.general_reading(user_select,cards)
        print("response: ",response.get_json())
        return response

    except Exception as e:
        print("Error",str(e))
        return jsonify({"error":str(e)}),500

if __name__ == "__main__":
    app.run(port=7000)

c:\Users\SMHRD\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [09/Sep/2024 11:02:23] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '취업준비가 힘들다'}
Processing input: user_nick=12, user_input=취업준비가 힘들다


c:\Users\SMHRD\MindRoad\flask_modules\chatbot.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  ck_chain = LLMChain(llm=self.model, prompt=template)
c:\Users\SMHRD\MindRoad\flask_modules\chatbot.py:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = ck_chain.run(user_input=user_input).strip()


...

Received data: {'user_nick': '12', 'input': '지룬이~'}
Processing input: user_nick=12, user_input=지룬이~
Is problem flag: False


127.0.0.1 - - [09/Sep/2024 11:22:09] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [09/Sep/2024 11:22:15] "OPTIONS /chatbot HTTP/1.1" 200 -


Received data: {'user_nick': '12', 'input': '취업준비가 고민이야'}
Processing input: user_nick=12, user_input=취업준비가 고민이야
Is problem flag: True
고민임
Extracted reason: '취업/진로'
Problem reason: 취업/진로
reference : 님께서 왜 내가 취업이 이렇게 싫은지, 취업이라고 하면 떠오르는 이미지는 무엇인지에 대해 먼저 정리해 보시면 좋을 것 같습니다. 나에게 취업이란, 직장이란, 어떤 느낌이기에 좋은 조건의 면접이 와도 가고 싶지 않은 마음이 드는지, 가지 않기로 결정했을 때 어떤 감정들이 들었는지요.  하고 싶지 않은 마음에 대해 스스로를 비난하거나 채찍질을 하는 것이 아니라, 이렇게 불안함에도 불구하고, 취준 생활이 길어지는 것이 답답함에도 불구하고, 왜 나는 면접을 가고 싶지 않은지, 왜 일을 시작하고 싶지 않은지, 왜 돈을 벌고 싶지 않은지에 대해 그 이유를 찬찬히, 조심스럽게 들여다 보시는 것이 필요합니다.  이 과정이 필요한 건, 지금의   님께서 자격증도 따고, 이력서도 넣었음에도 불구하고 그 마지막 스텝이 내딛어지지 않는, 그 이유를 알게 되어야 그 부분에 대한 대처 방법도 마련할 수 있기 때문이에요. 우울감으로 무언가를 시작하기에 기력이 없는 것인지, 새로운 시작이 두려운 것인지, 낯선 사람과 만나는 게 버거운지... 등에 대해 솔직하게 내 마음을 들여다 봐 주세요.


127.0.0.1 - - [09/Sep/2024 11:22:38] "POST /chatbot HTTP/1.1" 200 -


정렬된 형태소 분석 결과: ['취업/NNG', '준비/NNG', '가/JKS', '고민/NNG', '이야/JKS']
